In [20]:
%load_ext autoreload
%autoreload 2

import time
from typing import Optional, Tuple, List, Dict, Any
import frame.frame_assembler as fa
import frame.frame as fr
import timeseries as ts
import histogram as hg
import plots as pt
from datetime import datetime
from elasticsearch import Elasticsearch
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import frame.frame_collection_stats as fcs
from lib import ESQuery

client = Elasticsearch("http://localhost:9200/", api_key="WGRuTF9wUUIzYVpjeXh5Wnl2RlA6Q3ZPUFJoZXRUX1NiX3NWQ0FGbHZEdw==")

def print_json(j):
    print(json.dumps(j, indent=4))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
index_datetime = "2025-03-02"
time_from = f"{index_datetime}T03:54:40.0Z"
time_to = f"{index_datetime}T03:55:00.0Z"
index = f"jaeger-span-{index_datetime}*"
namespace = "hifi_1"

frame_asm = fa.FrameAssembler(client, namespace, index_datetime, 10000)
musts = [{ "term": { "tag.frame_is_expired": "true" } }]
filters = add_range_to(time_from, time_to, [])
frame_asm.find_frames([], [], filters, index_datetime, 10000)
frame_asm.print_summary()

Finding Digitiser Event List
Finding process_digitiser_event_list_message
Finding process_kafka_message (digitiser-aggregator)
Finding process_digitiser_trace_message
Finding process_kafka_message (trace-to-events)
Finding process
Finding process_kafka_message (nexus-writer)
800 frames
800 process_kafka_messages_writer
6400 process_digitiser_event_list_messages
6400 process_kafka_messages_aggregator
6400 process_digitiser_trace_messages
6400 process_kafka_messages_events
6400 process_events


In [ ]:

'''
Extracting Incomplete Frames
'''

frames1 = fa.FrameAssembler(client, "hifi_1", "2025-02-24", 10000)
musts = [ {
    "function_score": {
        "random_score": {
            "seed": time.time_ns(),
            "field": "tag.metadata_timestamp"
        }
    }
}]
shoulds = []
filters = []
frames1.find_frames(musts, shoulds, filters, "2025-02-24", 20)

frames2 = fa.FrameAssembler(client, "hifi_1", "2025-02-24", 10000)
musts = [{ "term": { "tag.frame_is_expired": "true" } }]
shoulds = []
filters = []
frames2.find_frames(musts, shoulds, filters, "2025-02-24", 20)

frames1.print_summary()
frames2.print_summary()


In [9]:
import frame.frame_collection_stats as fcs
frames = [fr.Aggragator_Frame(frame,frame_asm) for frame in frame_asm.frames]
frames = fcs.FrameCollectionStats(frames)
frames.print("  ")


   [Child Digitisers]
   trace (us):               139 < 301.18515625 (~104.19991093692437) < 2226
   trace to eventlist (us):  605 < 4205.5771875 (~1665.698030753503) < 14481
   eventlist (us):           15 < 48.84796875 (~29.62253304089618) < 419
   trace to writer (ms):     16 < 38.63171875 (~10.823672789498286) < 68
   eventlist to writer (ms): 1 < 18.16015625 (~11.831861333825325) < 53


In [ ]:
import frame.frame_stats as fs
the_frames = [fr.Aggragator_Frame(frame,frames2) for frame in frames2.frames]
for frame in the_frames:
    #frame.print_summaries()
    stats = fs.FrameStats(frame)
    stats.print()